### Single model run
Optimise parameter set and do single model run with the resulting parameters. Then illustrate results with basic plots, including the matrix scaling.

In [ ]:
# Uncomment the following commands to install in Colab
# Once completed, click the 'Restart runtime' button that will appear to restart the Colab environment
# and then proceed to the following cells

# !pip uninstall numba -y
# !pip uninstall librosa -y
# !pip install estival==0.4.7b2 numpy==1.24.3 kaleido

In [ ]:
try:
    import google.colab
    on_colab = True
    ! git clone https://github.com/monash-emu/aust-covid.git --branch finalise-notebooks
    %cd aust-covid
    %pip install -e ./
    import multiprocessing as mp
    mp.set_start_method('forkserver')
except:
    on_colab = False

In [ ]:
import numpy as np
import pymc as pm

from estival.model import BayesianCompartmentalModel
from estival.wrappers import pymc as epm

from inputs.constants import SUPPLEMENT_PATH
from aust_covid.inputs import get_ifrs
from aust_covid.model import build_model
from aust_covid.calibration import get_priors, get_targets
from aust_covid.plotting import plot_single_run_outputs, plot_example_model_matrices
from emutools.tex import DummyTexDoc, StandardTexDoc
from emutools.parameters import load_param_info
from emutools.calibration import round_sigfig
from aust_covid.utils import add_image_to_doc

In [ ]:
max_iterations = 1000

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'single_run', 'Single run', 'austcovid')
param_info = load_param_info()
param_info['value'].update(get_ifrs(DummyTexDoc()))
parameters = param_info['value'].to_dict()
epi_model = build_model(DummyTexDoc(), mobility_sens=True)
priors = get_priors(False, param_info['abbreviations'], DummyTexDoc())
prior_names = [p.name for p in priors]
targets = get_targets(DummyTexDoc())
bcm = BayesianCompartmentalModel(epi_model, parameters, priors, targets)

In [ ]:
with pm.Model() as pmc_model:
    start_params = {k: np.clip(v, *bcm.priors[k].bounds(0.99)) for k, v in parameters.items() if k in bcm.priors}
    variables = epm.use_model(bcm)
    map_params = pm.find_MAP(start=start_params, vars=variables, include_transformed=False, maxeval=max_iterations)
    map_params = {k: float(v) for k, v in map_params.items()}
print('Best candidate parameters found:')
for i_param, param in enumerate([p for p in map_params if '_dispersion' not in p]):
    print(f'   {param}: {round_sigfig(map_params[param], 4)} (within bound {priors[i_param].bounds()}')
parameters.update(map_params)
epi_model.run(parameters=parameters)

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'single_run', 'Single run', 'austcovid')

In [ ]:
parameters.update(map_params)
epi_model.run(parameters=parameters)

In [ ]:
outputs_fig = plot_single_run_outputs(epi_model, targets)
outputs_fig

In [ ]:
example_matrix_fig = plot_example_model_matrices(epi_model, parameters)
example_matrix_fig

In [ ]:
add_image_to_doc(outputs_fig.update_layout(showlegend=False), 'single_run_outputs', 'svg', 'Outputs from single model run', app_doc, 'Outputs')
add_image_to_doc(example_matrix_fig, 'example_matrices', 'svg', 'Dynamic mixing matrices', app_doc, 'Mixing')

In [ ]:
app_doc.write_doc()

In [ ]:
if on_colab:
    # To build a PDF, we need the appropriate tex packages installed
    ! apt-get -y install texlive-latex-base texlive-fonts-recommended texlive-fonts-extra texlive-latex-extra texlive-bibtex-extra biber
    # To avoid clutter and navigate to the right directory
    import os
    os.chdir(SUPPLEMENT_PATH)
    # And finally build the formatted PDF, repeated commands are necessary
    ! pdflatex model_construction
    ! biber model_construction
    ! pdflatex model_construction
    ! pdflatex model_construction